In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
import os
import gc
import cv2
import time
import tqdm
import random
import collections
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tq
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# ablumentations for easy image augmentation for input as well as output
import albumentations as albu
# from albumentations import torch as AT
plt.style.use('bmh')

In [ ]:

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
use_gpu = torch.cuda.is_available()
SEED = 42

In [ ]:
X= ['{}'.format(num) for num in range(1, 2881)]
y= ['{}'.format(num) for num in range(1, 2881)]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
train=pd.DataFrame()
test=pd.DataFrame()

In [ ]:
train['x_train']=X_train
train['y_train']=y_train
test['x_test']=X_test
test['y_test']=y_test

In [ ]:
from skimage.io import imread, imsave
import skimage.io as io

In [ ]:
class Super_resol_Dataset():
    
    def __init__(self, dataset, root_dir_x, root_dir_y, transform1=None, transform2=None):

        self.data = dataset
        self.root_dir_x = root_dir_x
        self.root_dir_y = root_dir_y
        self.transform1 = transform1
        self.transform2 = transform2

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx
        # idx = idx
        img_name = self.root_dir_x + self.data.iloc[idx, 0]
        image = Image.open(img_name+".png")

        label_name =  self.root_dir_y +self.data.iloc[idx, 1]
        label = Image.open(label_name+".png")        
        if self.transform1 is not None:
            image = self.transform1(image)
        if self.transform2 is not None:
            label = self.transform2(label)

        return image, label
    


In [ ]:
path1='../input/sem-data/sem_128/image/'
path2='../input/sem-data/sem_128/label/'

In [ ]:
batch_size=32
train_transform = transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])

trainset = Super_resol_Dataset(dataset=train,root_dir_x=path1,
                               root_dir_y=path2,transform1=train_transform,transform2=test_transform)
testset = Super_resol_Dataset(dataset=test,root_dir_x=path1,
                               root_dir_y=path2,transform1=train_transform,transform2=test_transform)


In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(testset,batch_size=batch_size, shuffle=False, num_workers=1 )

In [ ]:
len(trainset)

In [ ]:

trainiter = iter(trainloader)
features, labels = next(trainiter)
features.shape, labels.shape

In [ ]:
use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:
save_file_name = 'segmentation_128.pt'

In [ ]:
import torch
from torch.autograd import Function
class DiceCoeff(Function):
    """Dice coeff for individual examples"""

    def forward(self, input, target):
        self.save_for_backward(input, target)
        eps = 0.0001
        self.inter = torch.dot(input.view(-1), target.view(-1))
        self.union = torch.sum(input) + torch.sum(target) + eps

        t = (2 * self.inter.float() + eps) / self.union.float()
        return t

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):

        input, target = self.saved_variables
        grad_input = grad_target = None

        if self.needs_input_grad[0]:
            grad_input = grad_output * 2 * (target * self.union - self.inter) \
                         / (self.union * self.union)
        if self.needs_input_grad[1]:
            grad_target = None

        return grad_input, grad_target


def dice_coeff(input, target):
    """Dice coeff for batches"""
    if input.is_cuda:
        s = torch.FloatTensor(1).cuda().zero_()
    else:
        s = torch.FloatTensor(1).zero_()

    for i, c in enumerate(zip(input, target)):
        s = s + DiceCoeff().forward(c[0], c[1])

    return s / (i + 1)

In [ ]:
!pip install pytorch-msssim

In [ ]:
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

In [ ]:

def training(model,criterion, optimizer,scheduler,n_epochs):
  # n_epochs = 32

    valid_loss_min = np.Inf # track change in validation loss
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        dice_train = 0.0
        dice_test = 0.0
        ssim_val= 0.0
        ms_ssim_val =0.0
      ###################
      # train the model #
      ###################
        model.train()
      # bar = tq(trainloader, postfix={"train_loss":0.0})
        for i,inputs in enumerate(trainloader):
            data,target = inputs
          # print(data.size(0))
            if use_gpu:data, target = data.cuda(), target.cuda()
          # clear the gradients of all optimized variables
            optimizer.zero_grad()
          # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
          # calculate the batch loss
            loss = criterion(output, target)
          #print(loss)
          # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
          # perform a single optimization step (parameter update)
            optimizer.step()
          # update training loss
            train_loss += loss.item()*data.size(0)
            output= torch.sigmoid(output)
            output = (output > 0.5).float()
            dice_cof = dice_coeff(output, target).item()
            dice_train +=  dice_cof * data.size(0)
          # bar.set_postfix(ordered_dict={"train_loss":loss.item()})
        
        model.eval()
      # del data, target
        with torch.no_grad():
            for i,inputs in enumerate(testloader):
                data,target = inputs
                if use_gpu:
                      data, target = data.cuda(), target.cuda()
              # forward pass: compute predicted outputs by passing inputs to the model
                output = model(data)
            # calculate the batch loss
                loss = criterion(output, target)
            # update average validation loss 
                valid_loss += loss.item()*data.size(0)
                output= torch.sigmoid(output)
                output = (output > 0.5).float()
                dice_cof = dice_coeff(output, target).item()
                dice_test +=  dice_cof * data.size(0)
                ssim_val =ssim_val+ ssim( output, target, data_range=1, size_average=True)* data.size(0)
#                 ms_ssim_val =ms_ssim_val+ ms_ssim( output, target, data_range=1, size_average=True )* data.size(0) 
            # bar.set_postfix(ordered_dict={"valid_loss":loss.item(), "dice_score":dice_cof})
    
        train_loss = train_loss/len(trainloader.dataset)
        valid_loss = valid_loss/len(testloader.dataset)
        dice_test = dice_test/len(testloader.dataset)
        dice_train = dice_train/len(trainloader.dataset)
        ssim_val = ssim_val/len(testloader.dataset)
#         ms_ssim_val = ms_ssim_val/len(testloader.dataset)

        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)
        dice_train_list.append(dice_train)
        dice_test_list.append(dice_test)
        lr_rate_list.append([param_group['lr'] for param_group in optimizer.param_groups])

      # print training/validation statistics 
        print('Epoch: {}  Training Loss: {:.6f}  Validation Loss: {:.6f} Dice test: {:.6f}   ssim Score: {:.6f}  dice train: {:.6f}'.format(
            epoch, train_loss, valid_loss, dice_test,ssim_val,dice_train))
      
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_file_name)
            valid_loss_min = valid_loss
      
        scheduler.step(valid_loss)

In [ ]:
from torchvision.transforms import *
# del model
model = UNet(n_channels=1, n_classes=1, bilinear=True)

if use_gpu:
    print('GPU is avaialble!')
    model = model.cuda()

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.99)
current_lr = [param_group['lr'] for param_group in optimizer.param_groups][0]
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',factor=0.2, patience=2)

In [ ]:
train_loss_list = []
valid_loss_list = []
dice_train_list = []
dice_test_list = []
lr_rate_list = []

In [ ]:
training(model, criterion,optimizer,scheduler,n_epochs=40)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot([i[0] for i in lr_rate_list])
plt.ylabel('learing rate during training', fontsize=22)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(train_loss_list,  marker='o', label="Training Loss")
plt.plot(valid_loss_list,  marker='o', label="Validation Loss")
plt.ylabel('loss', fontsize=22)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(dice_train_list,  marker='o', label="Training Dice Score")
plt.plot(dice_test_list,  marker='o', label="Validation Dice Score")
plt.ylabel('Dice Coefficient', fontsize=22)
plt.legend()
plt.show()

In [ ]:
# load best model
model.load_state_dict(torch.load('./segmentation_128.pt'))
model.cuda()
model.eval();

In [ ]:
def dice(img1, img2):
    img1 = np.asarray(img1).astype(np.bool)
    img2 = np.asarray(img2).astype(np.bool)

    intersection = np.logical_and(img1, img2)

    return 2.0 * intersection.sum() / (img1.sum() + img2.sum())

In [ ]:
test_transform = transforms.Compose([transforms.Resize((1024,1024)),transforms.ToTensor()])

from PIL import Image, ImageOps  
from torch.autograd import Variable

def image_loader(image_name):
    image = Image.open(image_name)
    image = test_transform(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0) 
    return image.cuda()


In [ ]:
#Testing image
inputs = image_loader('../input/sem-data/sem/sem/image/4.png')
output = model(inputs)
probs = torch.sigmoid(output)
probs = probs.squeeze(0)
tf = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Resize((1024,1024)),
                transforms.ToTensor()
            ]
        )

probs = tf(probs.cpu())
full_mask = probs.squeeze().cpu().numpy()
mask = full_mask>0.5
result=Image.fromarray((mask * 255).astype(np.uint8))

In [ ]:
truth = Image.open('../input/sem-data/sem/sem/label/4.png')

In [ ]:
result.save('result.png')

In [ ]:
print(dice(result,truth))